# Table tennis simulation

This example shows the usage of `DiscreteEvents.jl` with event driven state machines.

We implement players as timed state machines and thus need definitions of states and events and some data describing the players:

In [1]:
using DiscreteEvents, Random, Printf

abstract type PState end
struct Idle <: PState end
struct Wait <: PState end
struct Unalert <: PState end

abstract type PEvent end
struct Start <: PEvent end
struct Serve <: PEvent end
struct Return <: PEvent end
struct Miss <: PEvent end

mutable struct Player
    name::AbstractString
    opp::Union{Number,Player}
    state::PState
    accuracy::Float64
    attentiveness::Float64
    score::Int64

    Player(name, acc, att) = new(name, 0, Idle(), acc, att, 0)
end

Then we define some physical facts and a function to randomize them:

In [2]:
const dist = 3 # distance for ball to fly [m]
const vs   = 10 # serve velocity [m/s]
const vr   = 20 # return velocity [m/s]

rd(s::Float64) = randn()*s + 1

rd (generic function with 1 method)

Next we must describe the behaviour of our players. They are modeled as finite state machines, which have known states and react to known events. This is done with the `step!` function. Julia's multiple dispatch allows to give multiple definitions of `step!` for different combinations of states and events.

The `serve` and `ret`-functions, used for describing serving and return of players are used to randomize the time and the behaviour of players. The players thus act probabilistically as Markov automata.

In [3]:
function init!(p::Player, opp::Player)
    p.opp = opp
    p.state = rand() ≤ p.attentiveness ? Wait() : p.state = Unalert()
end

function serve(p::Player)
    ts = 3 + dist*rd(0.15)/(vs*rd(0.25))
    if (rand() ≤ p.accuracy) && (p.state == Wait())
        event!(fun(step!, p.opp, Serve()), after, ts)
        @printf("%5.2f: %s serves %s\n", tau()+ts, p.name, p.opp.name)
    else
        event!(fun(step!, p.opp, Miss()), after, ts)
        @printf("%5.2f: %s serves and misses %s\n", tau()+ts, p.name, p.opp.name)
    end
    rand() ≥ p.attentiveness && (p.state = Unalert())
end

function ret(p::Player)
    tr = dist*rd(0.15)/(vr*rd(0.25))
    if rand() ≤ p.accuracy
        event!(fun(step!, p.opp, Return()), after, tr)
        @printf("%5.2f: %s returns %s\n", tau()+tr, p.name, p.opp.name)
    else
        event!(fun(step!, p.opp, Miss()), after, tr)
        @printf("%5.2f: %s returns and misses %s\n", tau()+tr, p.name, p.opp.name)
    end
    rand() ≥ p.attentiveness && (p.state = Unalert())
end

ret (generic function with 1 method)

The actual behaviour of a player is implemented as a state machine via the `step!`--function.

In [4]:
"default transition for players"
step!(p::Player, q::PState, σ::PEvent) =
        println("undefined transition for $(p.name), $q, $σ")

"player p gets a start command"
step!(p::Player, ::Union{Wait, Unalert}, ::Start) = serve(p)

"player p is waiting and gets served or returned"
step!(p::Player, ::Wait, ::Union{Serve, Return}) = ret(p)

"player p is unalert and gets served or returned"
function step!(p::Player, ::Unalert, ::Union{Serve, Return})
    @printf("%5.2f: %s looses ball\n", tau(), p.name)
    p.opp.score += 1
    p.state = Wait()
    serve(p)
end

"player p is waiting or unalert and gets missed"
function step!(p::Player, ::Union{Wait, Unalert}, ::Miss)
    p.score += 1
    p.state = Wait()
    serve(p)
end

"simplified `step!` call"
step!(p::Player, σ::PEvent) = step!(p, p.state, σ)

step!

In order to setup a simulation, we have to create and initialize the players, to start and run the game:

In [5]:
ping = Player("Ping", 0.90, 0.90)
pong = Player("Pong", 0.90, 0.90)
init!(ping, pong)
init!(pong, ping)
step!(ping, Start())

Random.seed!(123)

println(run!(𝐶, 30))
println("Ping scored $(ping.score)")
println("Pong scored $(pong.score)")

 3.24: Ping serves Pong
 3.36: Pong returns Ping
 3.53: Ping returns Pong
 3.69: Pong returns Ping
 3.81: Ping returns Pong
 3.99: Pong returns Ping
 4.11: Ping returns Pong
 4.22: Pong returns Ping
 4.33: Ping returns Pong
 4.42: Pong returns Ping
 4.56: Ping returns Pong
 4.70: Pong returns Ping
 4.87: Ping returns Pong
 4.99: Pong returns Ping
 5.13: Ping returns Pong
 5.30: Pong returns Ping
 5.52: Ping returns Pong
 5.71: Pong returns Ping
 5.80: Ping returns Pong
 6.43: Pong returns Ping
 6.58: Ping returns Pong
 6.66: Pong returns Ping
 6.82: Ping returns Pong
 6.94: Pong returns Ping
 7.06: Ping returns Pong
 7.26: Pong returns Ping
 7.43: Ping returns and misses Pong
10.63: Pong serves and misses Ping
13.91: Ping serves Pong
14.08: Pong returns and misses Ping
17.38: Ping serves Pong
17.52: Pong returns Ping
17.64: Ping returns Pong
17.76: Pong returns Ping
18.04: Ping returns Pong
18.19: Pong returns Ping
18.35: Ping returns Pong
18.44: Pong returns Ping
18.44: Ping looses ba

Finally we reset `𝐶` for further simulations.

In [6]:
resetClock!(𝐶)

"clock reset to t₀=0.0, sampling rate Δt=0.01."